# Classification d'IRM Cérébrales : Tumeur vs Sans Tumeur
## CNN (Convolutional Neural Network) entraîné depuis zéro

**Dataset** : 253 images IRM cérébrales  
**Classes** : 
- `yes` : IRM avec tumeur
- `no` : IRM sans tumeur

**Objectif** : Atteindre >92% d'accuracy en validation

## 1. Importation des bibliothèques

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import cv2
from PIL import Image

# TensorFlow et Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

## 2. Configuration et paramètres

In [ ]:
# Paramètres
IMG_SIZE = 224  # Taille des images 224×224
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 0.001
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.1

# Chemins (à adapter selon votre structure)
DATA_DIR = 'brain_tumor_dataset'  # Changez ce chemin si nécessaire
YES_DIR = os.path.join(DATA_DIR, 'yes')
NO_DIR = os.path.join(DATA_DIR, 'no')
MODEL_SAVE_PATH = 'best_brain_tumor_model.keras'

# Seed pour reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

## 3. Chargement et exploration des données

In [ ]:
def load_images_from_folder(folder, label):
    """
    Charge les images depuis un dossier et les convertit en niveaux de gris 224×224
    
    Args:
        folder: chemin du dossier contenant les images
        label: étiquette (0 pour 'no', 1 pour 'yes')
    
    Returns:
        images: liste des images prétraitées
        labels: liste des étiquettes correspondantes
    """
    images = []
    labels = []
    
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            try:
                # Charger l'image en niveaux de gris
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
                if img is not None:
                    # Redimensionner à 224×224
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    images.append(img)
                    labels.append(label)
            except Exception as e:
                print(f"Erreur lors du chargement de {filename}: {e}")
    
    return images, labels

# Chargement des images
print("Chargement des images...")
images_no, labels_no = load_images_from_folder(NO_DIR, 0)
images_yes, labels_yes = load_images_from_folder(YES_DIR, 1)

# Combiner les données
X = np.array(images_no + images_yes)
y = np.array(labels_no + labels_yes)

print(f"\nNombre total d'images: {len(X)}")
print(f"Images sans tumeur (no): {len(images_no)}")
print(f"Images avec tumeur (yes): {len(images_yes)}")
print(f"Shape des images: {X.shape}")

In [ ]:
# Visualisation d'exemples d'images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Exemples d\'images du dataset', fontsize=16)

# Afficher 5 images sans tumeur
for i in range(5):
    axes[0, i].imshow(images_no[i], cmap='gray')
    axes[0, i].set_title('Sans tumeur')
    axes[0, i].axis('off')

# Afficher 5 images avec tumeur
for i in range(5):
    axes[1, i].imshow(images_yes[i], cmap='gray')
    axes[1, i].set_title('Avec tumeur')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Distribution des classes
plt.figure(figsize=(8, 5))
unique, counts = np.unique(y, return_counts=True)
plt.bar(['Sans tumeur (0)', 'Avec tumeur (1)'], counts, color=['green', 'red'], alpha=0.7)
plt.title('Distribution des classes', fontsize=14)
plt.ylabel('Nombre d\'images')
plt.xlabel('Classe')
for i, count in enumerate(counts):
    plt.text(i, count + 5, str(count), ha='center', fontsize=12, fontweight='bold')
plt.show()

## 4. Pré-traitement des données

In [ ]:
# Normalisation des pixels entre 0 et 1
X = X.astype('float32') / 255.0

# Ajouter une dimension pour le canal (grayscale)
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(f"Shape finale de X: {X.shape}")
print(f"Shape de y: {y.shape}")
print(f"Plage de valeurs des pixels: [{X.min()}, {X.max()}]")

In [ ]:
# Division en ensembles d'entraînement, validation et test
# D'abord, séparer le test set
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=TEST_SPLIT, random_state=42, stratify=y
)

# Ensuite, séparer train et validation
val_size_adjusted = VALIDATION_SPLIT / (1 - TEST_SPLIT)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=val_size_adjusted, random_state=42, stratify=y_temp
)

print(f"\nDivision des données:")
print(f"Entraînement: {X_train.shape[0]} images ({len(y_train[y_train==0])} sans tumeur, {len(y_train[y_train==1])} avec tumeur)")
print(f"Validation: {X_val.shape[0]} images ({len(y_val[y_val==0])} sans tumeur, {len(y_val[y_val==1])} avec tumeur)")
print(f"Test: {X_test.shape[0]} images ({len(y_test[y_test==0])} sans tumeur, {len(y_test[y_test==1])} avec tumeur)")

## 5. Data Augmentation

In [ ]:
# Configuration de l'augmentation des données pour l'entraînement
train_datagen = ImageDataGenerator(
    rotation_range=15,           # Rotation aléatoire jusqu'à 15°
    width_shift_range=0.1,       # Décalage horizontal
    height_shift_range=0.1,      # Décalage vertical
    shear_range=0.1,             # Cisaillement
    zoom_range=0.1,              # Zoom aléatoire
    horizontal_flip=True,        # Retournement horizontal
    fill_mode='nearest'          # Remplissage des pixels manquants
)

# Pas d'augmentation pour validation et test
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Générer les batches
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE)

print("Data augmentation configurée avec succès!")

In [ ]:
# Visualisation de l'augmentation des données
sample_img = X_train[0:1]
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Exemples de data augmentation', fontsize=16)

for i in range(10):
    augmented = train_datagen.flow(sample_img, batch_size=1)
    aug_img = next(augmented)[0]
    row = i // 5
    col = i % 5
    axes[row, col].imshow(aug_img.squeeze(), cmap='gray')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

## 6. Construction du modèle CNN personnalisé

**Architecture:**
- 3 blocs convolutifs (Conv2D + BatchNormalization + MaxPooling)
- Couches Dense avec Dropout
- Activation finale sigmoid pour classification binaire

In [ ]:
def create_cnn_model(input_shape=(IMG_SIZE, IMG_SIZE, 1)):
    """
    Crée un modèle CNN personnalisé pour la classification binaire
    
    Architecture:
    - Bloc 1: Conv2D(32) + BatchNorm + MaxPooling + Dropout
    - Bloc 2: Conv2D(64) + BatchNorm + MaxPooling + Dropout
    - Bloc 3: Conv2D(128) + BatchNorm + MaxPooling + Dropout
    - Flatten
    - Dense(128) + Dropout
    - Dense(64) + Dropout
    - Dense(1, sigmoid)
    """
    model = Sequential([
        # Bloc Convolutif 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Bloc Convolutif 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Bloc Convolutif 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Couches fully connected
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        # Couche de sortie (classification binaire)
        Dense(1, activation='sigmoid')
    ])
    
    return model

# Créer le modèle
model = create_cnn_model()

# Afficher l'architecture
model.summary()

In [ ]:
# Compilation du modèle
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print("Modèle compilé avec succès!")

## 7. Callbacks pour l'entraînement

In [ ]:
# Early Stopping: arrêter si pas d'amélioration
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Model Checkpoint: sauvegarder le meilleur modèle
model_checkpoint = ModelCheckpoint(
    MODEL_SAVE_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Reduce Learning Rate: réduire le taux d'apprentissage si plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

callbacks = [early_stopping, model_checkpoint, reduce_lr]

print("Callbacks configurés:")
print("- Early Stopping (patience=10)")
print("- Model Checkpoint (meilleure val_accuracy)")
print("- Reduce Learning Rate on Plateau")

## 8. Entraînement du modèle

In [ ]:
# Calculer les steps par epoch
steps_per_epoch = len(X_train) // BATCH_SIZE
validation_steps = len(X_val) // BATCH_SIZE

print(f"Début de l'entraînement...")
print(f"Steps par epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}\n")

# Entraîner le modèle
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Entraînement terminé!")

## 9. Visualisation des résultats d'entraînement

In [ ]:
# Graphiques de l'accuracy et de la loss
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_title('Accuracy pendant l\'entraînement', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_title('Loss pendant l\'entraînement', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Afficher les meilleures métriques
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
print(f"\n🏆 Meilleure validation accuracy: {best_val_acc*100:.2f}% (epoch {best_epoch})")

## 10. Évaluation sur l'ensemble de test

In [ ]:
# Charger le meilleur modèle sauvegardé
best_model = load_model(MODEL_SAVE_PATH)
print("✓ Meilleur modèle chargé depuis:", MODEL_SAVE_PATH)

# Évaluation sur le test set
test_loss, test_accuracy, test_precision, test_recall = best_model.evaluate(
    X_test, y_test, verbose=0
)

# Calcul du F1-Score
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"\n{'='*50}")
print(f"RÉSULTATS SUR L'ENSEMBLE DE TEST")
print(f"{'='*50}")
print(f"Accuracy:  {test_accuracy*100:.2f}%")
print(f"Precision: {test_precision*100:.2f}%")
print(f"Recall:    {test_recall*100:.2f}%")
print(f"F1-Score:  {f1_score*100:.2f}%")
print(f"Loss:      {test_loss:.4f}")
print(f"{'='*50}")

In [ ]:
# Prédictions sur le test set
y_pred_proba = best_model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Rapport de classification détaillé
print("\nRapport de classification détaillé:")
print(classification_report(
    y_test, y_pred, 
    target_names=['Sans tumeur (0)', 'Avec tumeur (1)']
))

## 11. Matrice de confusion et courbe ROC

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Matrice de confusion
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Sans tumeur', 'Avec tumeur'],
            yticklabels=['Sans tumeur', 'Avec tumeur'],
            ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_title('Matrice de Confusion', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Vraie classe')
axes[0].set_xlabel('Classe prédite')

# Courbe ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

axes[1].plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC curve (AUC = {roc_auc:.3f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Taux de Faux Positifs')
axes[1].set_ylabel('Taux de Vrais Positifs')
axes[1].set_title('Courbe ROC', fontsize=14, fontweight='bold')
axes[1].legend(loc="lower right")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAUC-ROC Score: {roc_auc:.4f}")

## 12. Prédiction en temps réel sur nouvelles IRM

In [ ]:
def predict_brain_tumor(model, image_path):
    """
    Prédit si une IRM cérébrale contient une tumeur
    
    Args:
        model: modèle CNN entraîné
        image_path: chemin vers l'image IRM
    
    Returns:
        prediction: probabilité de présence de tumeur
        label: 'Tumeur détectée' ou 'Pas de tumeur'
    """
    # Charger et prétraiter l'image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.astype('float32') / 255.0
    img = img.reshape(1, IMG_SIZE, IMG_SIZE, 1)
    
    # Prédiction
    prediction = model.predict(img, verbose=0)[0][0]
    
    # Interprétation
    if prediction > 0.5:
        label = "⚠️ TUMEUR DÉTECTÉE"
        confidence = prediction * 100
    else:
        label = "✓ PAS DE TUMEUR"
        confidence = (1 - prediction) * 100
    
    return prediction, label, confidence

# Test sur quelques images de test
num_samples = 6
indices = np.random.choice(len(X_test), num_samples, replace=False)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, i in enumerate(indices):
    img = X_test[i]
    true_label = y_test[i]
    
    # Prédiction
    pred_proba = best_model.predict(img.reshape(1, IMG_SIZE, IMG_SIZE, 1), verbose=0)[0][0]
    pred_label = 1 if pred_proba > 0.5 else 0
    
    # Affichage
    axes[idx].imshow(img.squeeze(), cmap='gray')
    
    true_text = "Tumeur" if true_label == 1 else "Sans tumeur"
    pred_text = "Tumeur" if pred_label == 1 else "Sans tumeur"
    color = 'green' if pred_label == true_label else 'red'
    
    axes[idx].set_title(
        f"Vrai: {true_text}\nPrédit: {pred_text} ({pred_proba*100:.1f}%)",
        color=color, fontweight='bold'
    )
    axes[idx].axis('off')

plt.suptitle('Prédictions sur nouvelles IRM', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Fonction pour prédire sur une nouvelle image (à utiliser avec un fichier externe)
def predict_new_image(image_path):
    """
    Fonction pratique pour prédire sur une nouvelle IRM
    
    Usage:
        predict_new_image('path/to/brain_scan.jpg')
    """
    prediction, label, confidence = predict_brain_tumor(best_model, image_path)
    
    # Afficher l'image et le résultat
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(img, cmap='gray')
    plt.title(f"{label}\nConfiance: {confidence:.2f}%", 
              fontsize=14, fontweight='bold',
              color='red' if prediction > 0.5 else 'green')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"\n{label}")
    print(f"Probabilité de tumeur: {prediction*100:.2f}%")
    print(f"Confiance: {confidence:.2f}%")
    
    return prediction, label, confidence

# Exemple d'utilisation:
# predict_new_image('chemin/vers/nouvelle_irm.jpg')

print("\n✓ Fonction de prédiction prête à l'emploi!")
print("  Usage: predict_new_image('path/to/your/brain_scan.jpg')")

## 13. Sauvegarde et résumé final

In [ ]:
# Résumé final
print("="*60)
print("RÉSUMÉ DU PROJET - CLASSIFICATION D'IRM CÉRÉBRALES")
print("="*60)
print(f"\n📊 DATASET:")
print(f"   - Total d'images: {len(X)}")
print(f"   - Images avec tumeur: {len(images_yes)}")
print(f"   - Images sans tumeur: {len(images_no)}")
print(f"   - Taille des images: {IMG_SIZE}×{IMG_SIZE} pixels (niveaux de gris)")

print(f"\n🏗️ ARCHITECTURE:")
print(f"   - 3 blocs convolutifs (Conv2D + BatchNorm + MaxPooling + Dropout)")
print(f"   - Filtres: 32 → 64 → 128")
print(f"   - Couches Dense: 128 → 64 → 1 (sigmoid)")
print(f"   - Total de paramètres: {model.count_params():,}")

print(f"\n🔧 ENTRAÎNEMENT:")
print(f"   - Epochs: {len(history.history['loss'])}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Learning rate initiale: {LEARNING_RATE}")
print(f"   - Data augmentation: ✓")
print(f"   - Early stopping: ✓")
print(f"   - Model checkpoint: ✓")

print(f"\n📈 PERFORMANCES:")
print(f"   - Test Accuracy: {test_accuracy*100:.2f}%")
print(f"   - Test Precision: {test_precision*100:.2f}%")
print(f"   - Test Recall: {test_recall*100:.2f}%")
print(f"   - F1-Score: {f1_score*100:.2f}%")
print(f"   - AUC-ROC: {roc_auc:.4f}")

objectif_atteint = "✓ OUI" if test_accuracy > 0.92 else "✗ NON"
print(f"\n🎯 OBJECTIF (>92% accuracy): {objectif_atteint}")

print(f"\n💾 MODÈLE SAUVEGARDÉ:")
print(f"   - Fichier: {MODEL_SAVE_PATH}")
print(f"   - Taille: {os.path.getsize(MODEL_SAVE_PATH) / (1024*1024):.2f} MB")

print(f"\n🚀 PRÉDICTION EN TEMPS RÉEL:")
print(f"   - Fonction disponible: predict_new_image()")
print(f"   - Input: Image IRM (JPG/PNG)")
print(f"   - Output: Probabilité de tumeur + visualisation")

print("\n" + "="*60)
print("✓ PROJET TERMINÉ AVEC SUCCÈS!")
print("="*60)

## 14. Instructions d'utilisation

### Pour prédire sur une nouvelle IRM:

```python
# Méthode 1: Utiliser la fonction predict_new_image
predict_new_image('chemin/vers/votre_irm.jpg')

# Méthode 2: Chargement manuel du modèle
from tensorflow.keras.models import load_model
model = load_model('best_brain_tumor_model.keras')
prediction, label, confidence = predict_brain_tumor(model, 'votre_irm.jpg')
```

### Réentraînement avec nouveaux paramètres:

Modifiez les paramètres dans la section 2, puis relancez les cellules à partir de la section 6.

### Amélioration possible:

- Augmenter le dataset (collecte de plus d'images)
- Essayer Transfer Learning (VGG16, ResNet, EfficientNet)
- Ajuster les hyperparamètres (learning rate, batch size, dropout)
- Ajouter plus de couches convolutives
- Utiliser des techniques d'ensemble (voting, stacking)